In [12]:
#Importamos las librerias que se necesitan

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import sklearn

In [20]:
#Librerias para graficar
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn import datasets

In [14]:
#Importamos el dataset de cancer de mama

from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()

In [15]:
#Dividimos los datos en prueba y testeo

x = data.data
y = data.target
import numpy as np
def split_data(x, y):
    _split_ones = np.round(np.where(y == 1.0)[0].shape[0]*0.5, decimals=0).astype(int)
    _split_zeros = np.round(np.where(y == 0.0)[0].shape[0]*0.5, decimals=0).astype(int)
    training_indexes = np.concatenate([np.where(y == 1.0)[0][:_split_ones], np.where(y == 0.0)[0][:_split_zeros]], axis=0)
    testing_indexes = np.concatenate([np.where(y == 1.0)[0][_split_ones:], np.where(y == 0.0)[0][_split_zeros:]], axis=0)
    return x[training_indexes, :], y[training_indexes], x[testing_indexes, :], y[testing_indexes]

_x_train, _y_train, _x_test, _y_test = split_data(x, y)

In [16]:
print(x_train.shape)
print(y_train.shape) 
print(x_test.shape)
print(y_test.shape)

torch.Size([284, 30])
torch.Size([284])
torch.Size([285, 30])
torch.Size([285])


In [21]:
#Llamamos a la clase Model que representa nuestra red neuronal artificial

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class Model(nn.Module):
    def __init__(self, input_dimensions, hidden_dimensions, output_labels, feature_names, input_layer=None, output_layer=None):
        super().__init__()
        self.hidden_dimensions = hidden_dimensions
        self.output_labels = output_labels
        self.feature_names = feature_names
        self.input_layer = nn.Linear(input_dimensions, self.hidden_dimensions) if input_layer is None else input_layer
        self.output_layer = nn.Linear(self.hidden_dimensions, np.unique(self.output_labels).shape[0]) if output_layer is None else output_layer

    def forward(self, x):
        return F.softmax(self.output_layer(F.relu(self.input_layer(x))), dim=1)

    def get_weights(self):
        return self.input_layer.weight.detach().numpy(), self.output_layer.weight.detach().numpy()

    def find_significant_features(self, top_num_features=1):
        w, _ = self.get_weights()
        max_contribution_per_feature = np.reshape(np.max(w, axis=1), [w.shape[0]])
        return np.argsort(max_contribution_per_feature)[-top_num_features:]

ANN = Model(_x_train.shape[1], hidden_dimensions=30, output_labels=_y_train, feature_names=data['feature_names'])
optimizer = torch.optim.Adam(ANN.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

training_iterations  = 1000
x_train = Variable(torch.from_numpy(_x_train)).float()
y_train = Variable(torch.from_numpy(_y_train)).long()
x_test = Variable(torch.from_numpy(_x_test)).float()
y_test = Variable(torch.from_numpy(_y_test)).long()

loss_list = np.zeros(training_iterations).astype(float)
accuracy_list = np.zeros(training_iterations).astype(float)

In [22]:
#Esta es la arquitectura de la red neuronal artificial
ANN

Model(
  (input_layer): Linear(in_features=30, out_features=30, bias=True)
  (output_layer): Linear(in_features=30, out_features=2, bias=True)
)

In [23]:
for _iteration in range(training_iterations):
    y_pred = ANN(x_train)
    loss = loss_fn(y_pred, y_train)
    loss_list[_iteration] = loss.item()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    with torch.no_grad():
        y_pred = ANN(x_test)
        correct = (torch.argmax(y_pred, dim=1) == y_test).type(torch.FloatTensor)
        accuracy_list[_iteration] = correct.mean()